In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

In [2]:
sent_lvl_cov_df = pd.read_csv("model_output_dataset/coverage_measure_output_sentences.csv")
txt_lvl_cov_df = pd.read_csv("model_output_dataset/coverage_measure_output_text.csv")

sent_lvl_relv_df = pd.read_csv("model_output_dataset/sentences_lvl_relevance_score_data.csv")
txt_lvl_relv_df = pd.read_csv("model_output_dataset/text_lvl_relevance_score_data.csv")

## Rank the video again their main topics from combo of coverage + relevance score

As we use te result of coverage to filter out some of the high quality video content to train set, we now need to filter out those videos for apple-to-apple ranking results

In [3]:
print(txt_lvl_cov_df['Video_ID'].drop_duplicates().shape[0])
print(txt_lvl_relv_df['Video_ID'].drop_duplicates().shape[0])

90
72


In [4]:
txt_lvl_cov_df_validate = pd.merge(txt_lvl_cov_df, txt_lvl_relv_df['Video_ID'].drop_duplicates(), 'inner', left_on = ['Video_ID'], right_on = ['Video_ID'])

In [5]:
main_topics_lst = list(txt_lvl_relv_df['main_topic'].unique())

for topic in main_topics_lst:
    
    print()
    print('--------------------------')
    print("Under a main_topic {}, top 10 videos\nranked by total_relevance_score: \n".format(topic))
    # rank top 10 video in ters of relevance
    relev_video_ids = list(txt_lvl_relv_df[txt_lvl_relv_df['main_topic'] ==\
                                  topic][['Video_ID', 'main_topic', \
                                'total_txt_level_relevance_score']].drop_duplicates().sort_values(\
                                    by = 'total_txt_level_relevance_score', ascending = False).head(10)['Video_ID'])
    
    # rank top 10 video in ters of coverage
    cov_video_ids = list(txt_lvl_cov_df_validate[txt_lvl_cov_df_validate['main_topics'] == topic][['Video_ID', 'main_topics', \
                                'Avg_Cosine_Similarity']].drop_duplicates().sort_values(by = 'Avg_Cosine_Similarity', ascending = False).head(10)['Video_ID'])
    
    # print the result for comparison
    [print("Video rank {} -- By Relevance score: {} <--- VS --> By Coverage score: {} ".format(index+1, relev_video_ids[index],  cov_video_ids[index] )) for index in range(len(relev_video_ids) ) ]
    
    # print out the rank difference (comment videos rank discrepency) rank between the common top 10 videos measure by both model
    commen_vds = list(set(relev_video_ids).intersection(cov_video_ids))
    print()
    print("common top videos rank differences from top 10 ranks:")
    for com_vd in commen_vds:
        print("Video - {}: Relevance rank {} <-- VS --> Coverage rank {}".format(com_vd, relev_video_ids.index(com_vd) + 1, cov_video_ids.index(com_vd) + 1))
    
    


--------------------------
Under a main_topic insect_ant, top 10 videos
ranked by total_relevance_score: 

Video rank 1 -- By Relevance score: 9SPixBok5ls <--- VS --> By Coverage score: 9SPixBok5ls 
Video rank 2 -- By Relevance score: insects_intro_ant_v <--- VS --> By Coverage score: cXUCUvcscXs 
Video rank 3 -- By Relevance score: A_hEZNxG_H8 <--- VS --> By Coverage score: A_hEZNxG_H8 
Video rank 4 -- By Relevance score: vG-QZOTc5_Q <--- VS --> By Coverage score: insects_intro_ant_v 
Video rank 5 -- By Relevance score: 2S__fbCGwOM <--- VS --> By Coverage score: QNnmjyHPnbc 
Video rank 6 -- By Relevance score: HedZXw_hAbs <--- VS --> By Coverage score: kFiDThjUBTk 
Video rank 7 -- By Relevance score: pCxY70kPDnM <--- VS --> By Coverage score: NVT2vUQMKUc 
Video rank 8 -- By Relevance score: Rj67rHawDTg <--- VS --> By Coverage score: 2S__fbCGwOM 
Video rank 9 -- By Relevance score: QNnmjyHPnbc <--- VS --> By Coverage score: kBnmf1XlWdA 
Video rank 10 -- By Relevance score: 7_e0CA_nhaE

IndexError: list index out of range

In [ ]:
txt_relav_cov_merge_df = pd.merge(txt_lvl_relv_df[['Video_ID', 'main_topic', 'total_txt_level_relevance_score']].drop_duplicates(),
         txt_lvl_cov_df_validate[['Video_ID', 'main_topics', 'Avg_Cosine_Similarity']].drop_duplicates(),
        'left', left_on = ['Video_ID', 'main_topic'], right_on = ['Video_ID', 'main_topics'] ).drop("main_topic", axis = 1)

txt_relav_cov_merge_df.head()


In [ ]:
### rank by relevance and coverage score at the same time 

main_topics_lst = list(txt_relav_cov_merge_df['main_topics'].unique())

for topic in main_topics_lst:
    print()
    print('--------------------------')
    print("Under a main_topic {}, top 10 videos\nranked by both coverage then relevance: \n".format(topic))
    # rank top 10 video in ters of relevance
    topic_video_ids = list(txt_relav_cov_merge_df[txt_relav_cov_merge_df['main_topics'] ==\
                                  topic].drop_duplicates().sort_values(\
                                    by = ['Avg_Cosine_Similarity', 'total_txt_level_relevance_score' ], ascending = False).head(10)['Video_ID'])
    # print the result 
    [print("Video rank {} -- By Coverage then Relevance score: {}".format(index+1, topic_video_ids[index] )) for index in range(len(topic_video_ids) ) ]
    

In [ ]:
### rank by avg of relevance score + coverage score 

txt_relav_cov_merge_df['text_lvl_relevance_coverage_mean_score'] = txt_relav_cov_merge_df[['total_txt_level_relevance_score','Avg_Cosine_Similarity']].mean(axis=1)

main_topics_lst = list(txt_relav_cov_merge_df['main_topics'].unique())

for topic in main_topics_lst:
    print()
    print('--------------------------')
    print("Under a main_topic {}, top 10 videos (by video ids) \nranked by both coverage & relevance: \n".format(topic))
    # rank top 10 video in ters of relevance
    topic_video_ids = list(txt_relav_cov_merge_df[txt_relav_cov_merge_df['main_topics'] ==\
                                  topic].drop_duplicates().sort_values(\
                                    by = ['text_lvl_relevance_coverage_mean_score' ], ascending = False).head(10)['Video_ID'])
    # print the result 
    [print("Video rank {} -- By relevance_coverage_mean_score: {}".format(index+1, topic_video_ids[index] )) for index in range(len(topic_video_ids) ) ]
    


## Rank the Sentences from combo of coverage + relevance score

As we use te result of coverage to filter out some of the high quality video content to train set, we now need to filter out those videos for apple-to-apple ranking results

In [ ]:
sentence_lvl_cov_df_validate = pd.merge(sent_lvl_cov_df, sent_lvl_relv_df['Video_ID'].drop_duplicates(), \
         'inner', left_on = ['Video_ID'], right_on = ['Video_ID'])

sentence_lvl_cov_df_validate.shape

In [ ]:
sentence_lvl_cov_df_validate.columns

In [ ]:
sentence_lvl_cov_df_validate.columns

Start to compare the validation the result between coverage vs relevance model in sentence level

given a video ID, what are their sentences quality rankings? relevance vs coverage

In [ ]:
# main_topics_lst = list(sent_lvl_relv_df['main_topic'].unique())
video_id = 'insects_intro_ant_v'

print('--------------------------')
print("Under a Video {}, top 10 sentences\nranked by Relevance Score: \n".format(video_id))
# rank top 10 video in ters of relevance
relev_sentences_rank = sent_lvl_relv_df[(sent_lvl_relv_df['Video_ID'] == video_id)][['final_corrected_version_sentences_txt', \
            'total_relevancy_score_proba_sentence',\
            'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec']].drop_duplicates().sort_values(\
                                by = 'total_relevancy_score_proba_sentence', ascending = False).head(10).reset_index(drop = True).reset_index().rename(columns = {"index" : "Rank"})

relev_sentences_rank["Rank_by_Relevance"] = relev_sentences_rank["Rank"]  + 1

# rank top 10 video in ters of coverage
cov_sentences_rank = sentence_lvl_cov_df_validate[(sentence_lvl_cov_df_validate['Video_ID'] == video_id)][['final_corrected_version_sentences_txt', \
        'Avg_Cosine_Similarity']].drop_duplicates().sort_values(by = 'Avg_Cosine_Similarity', ascending = False).head(10).reset_index(drop = True).reset_index().rename(columns = {"index" : "Rank"})

cov_sentences_rank["Rank_by_Coverage"] = cov_sentences_rank["Rank"]  + 1
# print(cov_sentences_rank["Rank"])

#     print(cov_sentences_rank)

# print the result for comparison
[print("\nSentences rank {}:\nBy Relevance score:\n{}\n <--- VS -->\nBy Coverage Score:\n {} ".format(index+1, relev_sentences_rank[relev_sentences_rank["Rank_by_Relevance"] == index + 1 ]['final_corrected_version_sentences_txt'].values[0],  cov_sentences_rank[cov_sentences_rank["Rank_by_Coverage"] == index + 1]['final_corrected_version_sentences_txt'].values[0] )) for index in range(10) ]

    

In [ ]:
# print out the rank difference (comment videos rank discrepency) rank between the common top 10 sentences measure by both models
common_sentences_df = pd.merge(relev_sentences_rank,cov_sentences_rank\
        , 'inner', left_on = ['final_corrected_version_sentences_txt'], right_on = ['final_corrected_version_sentences_txt'])
print("Common top videos rank differences from top 10 ranks:")
print()
commen_sents = list(common_sentences_df['final_corrected_version_sentences_txt'])
relev_sentences_ranks = list(common_sentences_df["Rank_by_Relevance"])
cov_sentences_ranks = list(common_sentences_df['Rank_by_Coverage'])
for com_sen in commen_sents:
    print("\nSentences :\n{}\nRelevance rank: {} <-- VS --> Coverage rank {}".format(com_sen, common_sentences_df[common_sentences_df.final_corrected_version_sentences_txt == com_sen]["Rank_by_Relevance"].values[0], common_sentences_df[common_sentences_df.final_corrected_version_sentences_txt == com_sen]["Rank_by_Coverage"].values[0]))

### rank the sentences when take the avg of relevance and coverage model with the time stamp

In [ ]:
sent_lvl_relv_cov_df = pd.merge(sent_lvl_relv_df, sentence_lvl_cov_df_validate, 'left', left_on = ['Video_ID','final_corrected_version_sentences_txt'], right_on = ['Video_ID','final_corrected_version_sentences_txt'] )

sent_lvl_relv_cov_df['mean_relevance_coverage_score'] = sent_lvl_relv_cov_df[['total_relevancy_score_proba_sentence', 'Avg_Cosine_Similarity']].mean(axis = 1)


In [ ]:
# main_topics_lst = list(sent_lvl_relv_df['main_topic'].unique())
video_id = 'insects_intro_ant_v'

print('--------------------------')
print("Under a Video {}, top 10 sentences\nranked by Relevance + Coverage Score: \n".format(video_id))
# rank top 10 video in ters of relevance
sentences_rank = sent_lvl_relv_cov_df[(sent_lvl_relv_cov_df['Video_ID'] == video_id)][['final_corrected_version_sentences_txt', \
            'mean_relevance_coverage_score',\
            'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec']].drop_duplicates().sort_values(\
                                by = 'mean_relevance_coverage_score', ascending = False).head(10).reset_index(drop = True).reset_index().rename(columns = {"index" : "Rank"})

sentences_rank["Rank"] = sentences_rank["Rank"]  + 1

# print the result for comparison
[print("\nRank: #{} Sentence: '{}'\nSentence start time: {}s\nSentences end time: {}s ".format(index+1, \
                                                                               sentences_rank[sentences_rank["Rank"] == index + 1 ]['final_corrected_version_sentences_txt'].values[0],\
                                                                                np.round(sentences_rank[sentences_rank["Rank"] == index + 1]['sentence_level_timstamp_min_sec'].values[0], 3),\
                                                                                    np.round(sentences_rank[sentences_rank["Rank"] == index + 1]['sentence_level_timstamp_max_sec'].values[0], 3) )) for index in range(10) ]

    

In [ ]:
sentences_rank